In [749]:
from skimage import io
import numpy as np
from libsvm.svmutil import *  
from libsvm.svm import *  
from sklearn import svm
from PIL import Image 
import time
import os
import sys
from sklearn import decomposition
from scipy.ndimage import binary_hit_or_miss

In [750]:
TrainingSampleNum = 2000 # 学習サンプル総数
TestSampleNum = 10000 # テストサンプル総数
ClassNum = 10 # クラス数（今回は10）
ImageSize = 28 # 画像サイズ（今回は縦横ともに28）
TrainingDataFile = './Images/TrainingSamples/{0:1d}-{1:04d}.png'
TestDataFile = './Images/TestSamples/{0:1d}-{1:04d}.png'
#OutFile = './Images/OutSamples/gray_{0:1d}-{1:04d}.png' 
#TrainingDataFile = './Images/OutSamples/gray_{0:1d}-{1:04d}.png'

In [751]:
#def pca(dataMat, dataNum):
    #k = 300
    #New_dataMat = np.zeros((dataNum, k))
    #pca = decomposition.PCA(n_components = k)
    #New_dataMat = pca.fit_transform(dataMat)
    #return New_dataMat

In [752]:
def Zhang_Suen_thinning(binary_image):
    # オリジナルの画像をコピー
    image_thinned = binary_image.copy()
    # 初期化します。この値は次のwhile文の中で除かれます。
    changing_1 = changing_2 = [1]
    while changing_1 or changing_2:
        # ステップ1
        changing_1 = []
        rows, columns = image_thinned.shape
        for x in range(1, rows - 1):
            for y in range(1, columns -1):
                p2, p3, p4, p5, p6, p7, p8, p9 = neighbour_points = neighbours(x, y, image_thinned)
                if (image_thinned[x][y] == 1 and
                    2 <= sum(neighbour_points) <= 6 and # 条件2
                    count_transition(neighbour_points) == 1 and # 条件3
                    p2 * p4 * p6 == 0 and # 条件4
                    p4 * p6 * p8 == 0): # 条件5
                    changing_1.append((x,y))
        for x, y in changing_1:
            image_thinned[x][y] = 0
        # ステップ2
        changing_2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns -1):
                p2, p3, p4, p5, p6, p7, p8, p9 = neighbour_points = neighbours(x, y, image_thinned)
                if (image_thinned[x][y] == 1 and
                    2 <= sum(neighbour_points) <= 6 and # 条件2
                    count_transition(neighbour_points) == 1 and # 条件3
                    p2 * p4 * p8 == 0 and # 条件4
                    p2 * p6 * p8 == 0): # 条件5
                    changing_2.append((x,y))
        for x, y in changing_2:
            image_thinned[x][y] = 0        
    return image_thinned


# 2値画像の黒を1、白を0とするように変換するメソッドです
#def black_one(binary):
   # bool_image = binary.astype(bool)
    #inv_bool_image = ~bool_image
    #return inv_bool_image.astype(int)

# 画像の外周を0で埋めるメソッドです
def padding_zeros(image):
    import numpy as np
    m,n = np.shape(image)
    padded_image = np.zeros((m+2,n+2))
    padded_image[1:-1,1:-1] = image
    return padded_image

# 外周1行1列を除くメソッドです。
def unpadding(image):
    return image[1:-1, 1:-1]

# 指定されたピクセルの周囲のピクセルを取得するメソッドです
def neighbours(x, y, image):
    return [image[x-1][y], image[x-1][y+1], image[x][y+1], image[x+1][y+1], # 2, 3, 4, 5
             image[x+1][y], image[x+1][y-1], image[x][y-1], image[x-1][y-1]] # 6, 7, 8, 9

# 0→1の変化の回数を数えるメソッドです
#def count_transition(neighbours):
    #neighbours += neighbours[:1]
    #return sum( (n1, n2) == (0, 1) for n1, n2 in zip(neighbours, neighbours[1:]) )

# 黒を1、白を0とする画像を、2値画像に戻すメソッドです
#def inv_black_one(inv_bool_image):
    #bool_image = ~inv_bool_image.astype(bool)
    #return bool_image.astype(int) * 255

In [753]:
def img2vector(imgFile, thres):
    img = Image.open(imgFile).convert('L')
    img_arr = np.array(img, 'i') # 20px * 20px 灰度图像
    img_normlization = np.zeros(img_arr.shape, dtype=np.uint8)
    for y in range(0, img_arr.shape[0]):
        for x in range(0, img_arr.shape[1]):
            if img_arr[y,x] < thres:
                img_normlization[y,x] = 1
            else:
                img_normlization[y,x] = 0
    #img_normlization = np.round(img_arr/255) # 对灰度值进行归一化
    
    img_arr2 = np.reshape(img_normlization, (1,-1))  # 1 * 784 矩阵
    
    #img_2 = padding_zeros(img_normlization)
    #img_3 = Zhang_Suen_thinning(img_2)
    #img_4 = unpadding(img_3)
    #img_arr2 = np.reshape(img_4, (1,-1))  # 1 * 784 矩阵
    return img_arr2

In [754]:
def read_and_convert(imgFileList, label):
    dataLabel = [] # 存放类标签
    dataNum = len(imgFileList)
    dataMat = np.zeros((dataNum, 784)) # dataNum * 784 的矩阵
    for i in range(dataNum):
        imgName = imgFileList[i]
        #imgName = get_img_name_str(imgNameStr)  # 得到 数字_实例编号.png
        #print("imgName: {}".format(imgName))
        #sclassTag = imgName.split(".")[0].split("-")[0] # 得到 类标签(数字)
        #print("classTag: {}".format(classTag))
        dataLabel.append(label)
        dataMat[i,:] = img2vector(imgName, 20)
        
    return dataMat, dataLabel

In [755]:
# 读取训练数据
def read_all_data():
    #cName = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    #flist = "./Images/TrainingSamples/0-{1:04d}.png"
    #flist = get_file_list(train_data_path)
    flist = [ None ] * 200
    flist_ = [ None ] * 200
    for sample in range(0, TrainingSampleNum // ClassNum):
        flist[sample] = TrainingDataFile.format(0, sample)
    dataMat, dataLabel = read_and_convert(flist, 0)
    for label in range(1, ClassNum):
        for sample in range(0, TrainingSampleNum // ClassNum):
            flist_[sample] = TrainingDataFile.format(label, sample)
        dataMat_, dataLabel_ = read_and_convert(flist_, label)
        dataMat = np.concatenate((dataMat, dataMat_), axis=0)
        dataLabel = np.concatenate((dataLabel, dataLabel_), axis=0)
    
    #dataNum = dataMat.shape[0]
    #dataMat = pca(dataMat, dataNum) #PCA
    print(dataMat.shape)
    print(len(dataLabel))
    #np.set_printoptions(threshold=np.inf)
    #print(dataMat)
    return dataMat, dataLabel

In [756]:
# create model
def create_svm(dataMat, dataLabel, decision):
    clf = svm.SVC(C=2, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape=decision, degree=3, gamma='auto', kernel='rbf', 
                  max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
    clf.fit(dataMat, dataLabel)
    return clf

In [757]:
#dataMat, dataLabel = read_all_data()
#pca = decomposition.PCA(n_components = 300 )
#New_dataMat = pca.fit(dataMat)
#clf = create_svm(New_dataMat, dataLabel, 'ovr')

In [758]:
# 对10个数字进行分类测试
def main():
    tflist = [ None ] * 1000
    labelName = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    tst = time.time()
    allErrCount = 0
    allErrorRate = 0.0
    allScore = 0.0
    results = np.zeros((ClassNum, ClassNum))
    # 訓練データを読み込み
    dataMat, dataLabel = read_all_data()
    #PCA
    pca = decomposition.PCA(n_components = 40).fit(dataMat)
    New_dataMat = pca.transform(dataMat)
    #SVM
    clf = create_svm(New_dataMat, dataLabel, 'ovo')
    #for tcn  in tcName:
        #testPath = "Mnist-image\\test\\" + tcn
        #print("class " + tcn + " path is: {}.".format(testPath))
        #tflist = get_file_list(testPath)
        #tflist
    #dataMat, dataLabel = read_all_data()
    #clf = create_svm(dataMat, dataLabel, decision='ovr')
    for label in range(0, ClassNum):
        for sample in range(0, TestSampleNum // ClassNum):
            tflist[sample] = TestDataFile.format(label, sample)
        tdataMat, tdataLabel = read_and_convert(tflist, label)
        print("test dataMat shape: {0}, test dataLabel len: {1} ".format(tdataMat.shape, len(tdataLabel)))
        
        #tdataNum = tdataMat.shape[0]
        tdataMat = pca.transform(tdataMat)  #PCA
        print(tdataMat.shape)
        print(len(tdataLabel))
        
        #print("test dataLabel: {}".format(len(tdataLabel)))
        pre_st = time.time()
        preResult = clf.predict(tdataMat)
        pre_et = time.time()
        print("Recognition  {0} spent {1:.4f}s.".format(label, (pre_et-pre_st)))
        #print("predict result: {}".format(len(preResult)))
        for x in preResult:
            results[label, x] += 1 #Confusion matrix
            
        errCount = len([x for x in preResult if x!=label])
        print("errorCount: {}.".format(errCount))
        allErrCount += errCount
        score_st = time.time()
        score = clf.score(tdataMat, tdataLabel)
        score_et = time.time()
        print("computing score spent {:.6f}s.".format(score_et-score_st))
        allScore += score
        print("score: {:.6f}.".format(score))
        print("error rate is {:.6f}.".format((1-score)))
        print("---------------------------------------------------------")


    tet = time.time()
    print("Testing All class total spent {:.6f}s.".format(tet-tst))
    #print("All error Count is: {}.".format(allErrCount))
    avgAccuracy = allScore/10.0
    print("Average accuracy is: {:.6f}.".format(avgAccuracy))
    #print("Average error rate is: {:.6f}.".format(1-avgAccuracy))
    print("= Confusion matrix ===========")
    for t_label in range(0, ClassNum):
        for m_label in range(0, ClassNum):
            print("{:04g}, ".format(results[t_label, m_label]), end="")
        print("")

In [759]:
main()

(2000, 784)
2000
test dataMat shape: (1000, 784), test dataLabel len: 1000 
(1000, 40)
1000
Recognition  0 spent 0.0878s.
errorCount: 1000.
computing score spent 0.088986s.
score: 0.000000.
error rate is 1.000000.
---------------------------------------------------------
test dataMat shape: (1000, 784), test dataLabel len: 1000 
(1000, 40)
1000
Recognition  1 spent 0.0880s.
errorCount: 994.
computing score spent 0.088134s.
score: 0.006000.
error rate is 0.994000.
---------------------------------------------------------
test dataMat shape: (1000, 784), test dataLabel len: 1000 
(1000, 40)
1000
Recognition  2 spent 0.0882s.
errorCount: 1000.
computing score spent 0.087840s.
score: 0.000000.
error rate is 1.000000.
---------------------------------------------------------
test dataMat shape: (1000, 784), test dataLabel len: 1000 
(1000, 40)
1000
Recognition  3 spent 0.0875s.
errorCount: 1000.
computing score spent 0.087947s.
score: 0.000000.
error rate is 1.000000.
----------------------